In [1]:
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split

In [2]:
raw_data = pd.read_csv("vihallu-warmup.csv")

print("Số dòng dữ liệu gốc:", len(raw_data))
raw_data.head()

Số dòng dữ liệu gốc: 198


,id,context,prompt,response,label
0,c480e0d2-72e5-47d6-baca-a884db935c8c,"Theo pháp lệnh Vincennes năm 1374, vương quốc ...",Quyền phản đối được khôi phục bởi Orléans đã g...,Quyền phản đối còn cho phép Nghị viện được bổ ...,extrinsic
1,8e225849-4ed1-4397-8519-2b4cfee0c7c6,Hệ thống đường biển xuất phát từ các cảng biển...,Những sông lớn nào là các tuyến đường thủy nội...,Các tuyến đường thủy nội địa huyết mạch chạy t...,intrinsic
2,c5a2aef8-4c3f-4bac-9fc3-814094578fc0,"Năm 1928, Bộ Giao thông khởi thảo kế hoạch côn...",Độ dài công lộ Hán Trung-Thất Bàn Quan là bao ...,Công lộ Hán Trung-Thất Bàn Quan dài hơn 150 km...,extrinsic
3,0bae8fee-cd84-4ec1-a324-0777f6fa7a32,Sự tiến hóa của giới thực vật đã theo xu hướng...,Vì thực vật hạt trần chiếm ưu thế trong việc t...,Thực vật hạt trần kém đa dạng hơn và hiếm gặp ...,no
4,a8dcd1ed-c9a1-4786-b97b-27244896ff95,"Kể từ những năm 1970, chính phủ cũng thực hiện...",Cacs loaji dược pham muôn vào thi trường My ph...,Các loại dược phẩm muốn vào thị trường Mỹ phải...,no


- Loại bỏ dữ liệu bị thiếu

- Xóa trùng lặp

- Chuẩn hóa văn bản

In [3]:
def clean_text(text):
    if pd.isna(text):
        return ""
    # Xóa ký tự đặc biệt, giữ lại chữ, số, dấu câu cơ bản
    text = re.sub(r"[^a-zA-Z0-9À-ỹ\s\.,!?]", "", str(text))
    # Chuẩn hóa khoảng trắng
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Áp dụng tiền xử lý
data = raw_data.dropna(subset=["prompt", "response"]).drop_duplicates()
data["context"] = data["context"].apply(clean_text)
data["prompt"] = data["prompt"].apply(clean_text)
data["response"] = data["response"].apply(clean_text)

print("Sau khi tiền xử lý:", len(data))
data.head()

Sau khi tiền xử lý: 198


,id,context,prompt,response,label
0,c480e0d2-72e5-47d6-baca-a884db935c8c,"Theo pháp lệnh Vincennes năm 1374, vương quốc ...",Quyền phản đối được khôi phục bởi Orléans đã g...,Quyền phản đối còn cho phép Nghị viện được bổ ...,extrinsic
1,8e225849-4ed1-4397-8519-2b4cfee0c7c6,Hệ thống đường biển xuất phát từ các cảng biển...,Những sông lớn nào là các tuyến đường thủy nội...,Các tuyến đường thủy nội địa huyết mạch chạy t...,intrinsic
2,c5a2aef8-4c3f-4bac-9fc3-814094578fc0,"Năm 1928, Bộ Giao thông khởi thảo kế hoạch côn...",Độ dài công lộ Hán TrungThất Bàn Quan là bao n...,"Công lộ Hán TrungThất Bàn Quan dài hơn 150 km,...",extrinsic
3,0bae8fee-cd84-4ec1-a324-0777f6fa7a32,Sự tiến hóa của giới thực vật đã theo xu hướng...,Vì thực vật hạt trần chiếm ưu thế trong việc t...,Thực vật hạt trần kém đa dạng hơn và hiếm gặp ...,no
4,a8dcd1ed-c9a1-4786-b97b-27244896ff95,"Kể từ những năm 1970, chính phủ cũng thực hiện...",Cacs loaji dược pham muôn vào thi trường My ph...,Các loại dược phẩm muốn vào thị trường Mỹ phải...,no


In [4]:
# Tách 80% train, 10% validation, 10% test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data["label"])
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42, stratify=test_data["label"])

print("Train size:", len(train_data))
print("Validation size:", len(val_data))
print("Test size:", len(test_data))

Train size: 158
Validation size: 20
Test size: 20


### Chuyển dữ liệu sang format conversation

In [5]:
def convert_to_conversation(df):
    formatted = []
    for _, row in df.iterrows():
        context = row.get("context", "")
        prompt = row.get("prompt", "")
        response = row.get("response", "")
        label = row.get("label", "")

        messages = []
        if context.strip():
            messages.append({"role": "system", "content": context})
        messages.append({"role": "user", "content": prompt})
        messages.append({"role": "assistant", "content": response})

        formatted.append({
            "id": row.get("id", ""),
            "messages": messages,
            "label": label
        })
    return formatted

train_formatted = convert_to_conversation(train_data)
val_formatted = convert_to_conversation(val_data)
test_formatted = convert_to_conversation(test_data)

print("Ví dụ 1 sample sau khi chuẩn hóa:")
print(json.dumps(train_formatted[0], indent=2, ensure_ascii=False))

Ví dụ 1 sample sau khi chuẩn hóa:
{
  "id": "0fa12b46-1bab-4631-859c-a385199befd8",
  "messages": [
    {
      "role": "system",
      "content": "Thành phố trải qua tái phát triển quy mô lớn trong các thập niên 1950 và 1960. Quá trình này gồm có các bất động sản cao tầng quy mô lớn như Castle Vale. Bull Ring được xây dựng lại và ga Birmingham New Street được tái phát triển. Trong các thập niên sau Chiến tranh thế giới thứ hai, thành phần dân tộc tại Birmingham có thay đổi quan trọng, do thành phố tiếp nhận các dòng di dân đến từ các quốc gia trong và ngoài Thịnh vượng chung. The citys population peaked in 1951 at 1,113,000 residents."
    },
    {
      "role": "user",
      "content": "Công trình nào được xây dựng lại vào thập niên 50 và 60 của thế kỉ XX, trong khi dân số thành phố đạt đỉnh vào năm 1970 với 1,113,000 cư dân?"
    },
    {
      "role": "assistant",
      "content": "Castle Vale được xây dựng lại trong thập niên 50 và 60, đồng thời dân số thành phố đạt đỉnh vào năm 1

### Lưu ra file JSONL

In [6]:
def save_jsonl(data, path):
    with open(path, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

save_jsonl(train_formatted, "train.jsonl")
save_jsonl(val_formatted, "val.jsonl")
save_jsonl(test_formatted, "test.jsonl")

print("Đã lưu train/val/test JSONL!")

Đã lưu train/val/test JSONL!


### Kiểm tra thử với HuggingFace Dataset

In [8]:
from datasets import Dataset

train_dataset = Dataset.from_json("train.jsonl")
val_dataset = Dataset.from_json("val.jsonl")
test_dataset = Dataset.from_json("test.jsonl")

print(train_dataset)
print(val_dataset)
print(test_dataset)

c:\Users\nghoo\AppData\Local\anaconda3\envs\hallucination_llm_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 158 examples [00:00, 16183.55 examples/s]
Generating train split: 20 examples [00:00, 2774.93 examples/s]
Generating train split: 20 examples [00:00, 2803.40 examples/s]

Dataset({
    features: ['id', 'messages', 'label'],
    num_rows: 158
})
Dataset({
    features: ['id', 'messages', 'label'],
    num_rows: 20
})
Dataset({
    features: ['id', 'messages', 'label'],
    num_rows: 20
})
